In [1]:
import numpy as np
import unittest
import heapq as hq
import weakref
import contextlib

In [2]:
class Config:
    enable_backprop = True

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

In [3]:
def numerical_diff(f, x, eps=1e-4):
    return (f(Variable(x.data+eps)).data - f(Variable(x.data-eps)).data) / (2*eps)

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

def no_grad():
    return using_config('enable_backprop', False)

In [4]:
class Variable:
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}는 지원하지 않습니다.')
        self.name = name
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace("\\n", "\\n"+" "*9)
        return f'variable({p})'

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        seen_set = set()
        funcs = []

        def add_func(f):
            if f not in seen_set:
                seen_set.add(f)
                hq.heappush(funcs, f)
        
        add_func(self.creator)

        while funcs:
            f = hq.heappop(funcs)
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx if x.grad is None else gx + x.grad
                if x.creator is not None:
                    add_func(x.creator)
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None

class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

            return outputs if len(outputs) > 1 else outputs[0]

    def __lt__(self, other): # this for generation heap
        if self.generation > other.generation:
            return True
        return False

    def __str__(self):
        return f'{self.generation} gen : [{__class__}], {id(self)}'

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, x):
        raise NotImplementedError()

In [13]:
class Neg(Function):
    def forward(self, x):
        return -1 * x
    def backward(self, gy):
        return -1*gy

class Pow(Function):
    def __init__(self, c):
        self.c = c
    def forward(self, x):
        return x**self.c
    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        return gy*c*x**(c-1)

class Square(Function):
    def forward(self, x):
        return x**2
    def backward(self, gy):
        x = self.inputs[0].data
        return 2 * x * gy

class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    def backward(self, gy):
        return np.exp(self.inputs.data) * gy

class Add(Function):
    def forward(self, x0, x1):
        return x0 + x1
    def backward(self, gy):
        return gy, gy

class Mul(Function):
    def forward(self, x0, x1):
        return x0 * x1
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return x1*gy, x0*gy

class Sub(Function):
    def forward(self, x0, x1):
        return x0 - x1
    def backward(self, gy):
        return gy, -gy

class Div(Function):
    def forward(self, x0, x1):
        return x0 / x1
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy/x1, (-x0/x1**2)*gy

def neg(x):
    return Neg()(x)
def pow(x, c):
    return Pow(c)(x)
def square(x):
    return Square()(x)
def exp(x):
    return Exp()(x)
def add(x, y):
    y = as_array(y)
    return Add()(x, y)
def mul(x, y):
    y = as_array(y)
    return Mul()(x, y)
def sub(x, y):
    y = as_array(y)
    return Sub()(x, y)
def rsub(x, y):
    y = as_array(y)
    return Sub()(y, x)
def div(x, y):
    y = as_array(y)
    return Div()(x, y)
def rdiv(x, y):
    y = as_array(y)
    return Div()(y, x)

Variable.__neg__ = neg
Variable.__pow__ = pow
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul
Variable.__sub__ = sub
Variable.__rsub__ = rsub
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

In [6]:
class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6)
        self.assertEqual(x.grad, expected)
    def test_gradient_check(self):
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)

In [7]:
print(id(hq))
d = hq.heapify([])
print(id(d))

140396939479136
9484816


In [8]:
x = Variable(np.array(2.0))
a = square(x)
b = square(a)
c = square(a)
y = add(b, c)
y.backward()
print(y.data, x.grad, a.grad)
print(x, a, b, c, y)

32.0 64.0 None
variable(2.0) variable(4.0) variable(16.0) variable(16.0) variable(32.0)


In [17]:
x = Variable(np.array(2.0))
y = x ** 3
y.backward()
print(y, x.grad)

variable(8.0) 12.0


In [10]:
with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

In [5]:
import numpy as np
from dezero import Variable

def goldstein(x, y):
    z = (1 + (x+y+1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)
z.backward()
print(x.grad, y.grad)

-5376.0 8064.0


In [23]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..')) # 환경변수(PATH) 설정

In [33]:
import os
import subprocess

def _dot_var(v, verbose=False):
    name = '' if v.name is None else v.name
    if verbose and v.data is not None:
        if v.name is not None:
            name += ':'
        name += str(v.shape) + ' ' + str(v.dtype)
    return f'{id(v)} [label="{name}", color=orange, style=filled]\n'

def _dot_func(f):
    txt = f'{id(f)} [label="{f.__class__.__name__}", color=lightblue, style=filled, shape=box]\n'
    
    dot_edge = '{} -> {}\n'
    for x in f.inputs:
        txt += dot_edge.format(id(x), id(f))
    for y in f.outputs:
        txt += dot_edge.format(id(f), id(y()))
    return txt

def get_dot_graph(output, verbose=True):
    txt = ''
    seen_set = set()
    funcs = []

    def add_func(f):
        if f not in seen_set:
            seen_set.add(f)
            funcs.append(f)
    
    add_func(output.creator)
    txt += _dot_var(output)

    while funcs:
        f = funcs.pop()
        txt += _dot_func(f)
        for x in f.inputs:
            txt += _dot_var(x, verbose=verbose)
            if x.creator is not None:
                add_func(x.creator)

    return 'digraph g {\n' + txt + '}'

def plot_dot_graph(output, verbose=True, to_file='dot.png'):
    dot_graph = get_dot_graph(output, verbose)
    tmp_dir = os.path.join(os.path.expanduser('~'), '.dezero')
    if not os.path.exists(tmp_dir):
        os.mkdir(tmp_dir)
    graph_path = os.path.join(tmp_dir, 'tmp_graph.dot')

    with open(graph_path, 'w') as f:
        f.write(dot_graph)
    
    extension = os.path.splitext(to_file)[1][1:]
    cmd = f'dot {graph_path} -T {extension} -o {to_file}'
    print(cmd)
    subprocess.run(cmd, shell=True)

print(get_dot_graph(z))
plot_dot_graph(z)

digraph g {
139866525213456 [label="", color=orange, style=filled]
139866525213936 [label="Mul", color=lightblue, style=filled, shape=box]
139866525325488 -> 139866525213936
139866525213792 -> 139866525213936
139866525213936 -> 139866525213456
139866525325488 [label="() float64", color=orange, style=filled]
139866525213792 [label="() float64", color=orange, style=filled]
139866525311424 [label="Add", color=lightblue, style=filled, shape=box]
139866525311232 -> 139866525311424
139866525213408 -> 139866525311424
139866525311424 -> 139866525213792
139866525311232 [label="() float64", color=orange, style=filled]
139866525213408 [label="() int64", color=orange, style=filled]
139866525311568 [label="Mul", color=lightblue, style=filled, shape=box]
139866525323520 -> 139866525311568
139866525311328 -> 139866525311568
139866525311568 -> 139866525311232
139866525323520 [label="() float64", color=orange, style=filled]
139866525311328 [label="() float64", color=orange, style=filled]
13986652531238

In [24]:
import os
os.path.dirname(__file__)

NameError: name '__file__' is not defined

## External Modules

In [2]:
!pip install memory_profiler
!apt install graphviz

[sudo] password for cod3holic: 
